# On-Time Flight Performance with Spark and Cosmos DB (Seattle)


## On-Time Flight Performance Background
This notebook provides an analysis of On-Time Flight Performance and Departure Delays data using GraphFrames for Apache Spark.  
* Original blog post: [On-Time Flight Performance with GraphFrames with Apache Spark Blog Post](https://databricks.com/blog/2016/03/16/on-time-flight-performance-with-graphframes-for-apache-spark.html)
* Original Notebook: [On-Time Flight Performance with GraphFrames with Apache Spark Notebook](http://cdn2.hubspot.net/hubfs/438089/notebooks/Samples/Miscellaneous/On-Time_Flight_Performance.html)

<img style="align: center;" src="https://github.com/dennyglee/databricks/blob/master/images/airports-d3-m.gif?raw=true">

Source Data: 
* [OpenFlights: Airport, airline and route data](http://openflights.org/data.html)
* [United States Department of Transportation: Bureau of Transportation Statistics (TranStats)](http://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236&DB_Short_Name=On-Time)
 * Note, the data used here was extracted from the US DOT:BTS between 1/1/2014 and 3/31/2014*

References:
* [GraphFrames User Guide](http://graphframes.github.io/user-guide.html)
* [GraphFrames: DataFrame-based Graphs (GitHub)](https://github.com/graphframes/graphframes)
* [D3 Airports Example](http://mbostock.github.io/d3/talk/20111116/airports.html)

## Spark to Cosmos DB  Connector
Connecting Apache Spark to Azure Cosmos DB accelerates your ability to solve your fast moving Data Sciences problems where your data can be quickly persisted and retrieved using Azure Cosmos DB's DocumentDB API.  With the Spark to Cosmos DB conector, you can more easily solve scenarios including (but not limited to) blazing fast IoT scenarios, update-able columns when performing analytics, push-down predicate filtering, and performing advanced analytics to data sciences against your fast changing data against a geo-replicated managed document store with guaranteed SLAs for consistency, availability, low latency, and throughput.   

The Spark to Cosmos DB connector utilizes the [Azure DocumentDB Java SDK](https://github.com/Azure/azure-documentdb-java) will utilize the following flow:

<img style="align: center;" src="https://raw.githubusercontent.com/dennyglee/notebooks/master/images/Azure-DocumentDB-Spark_Connector_600x266.png">



The data flow is as follows:

1. Connection is made from Spark master node to Cosmos DB gateway node to obtain the partition map. Note, user only specifies Spark and Cosmos DB connections, the fact that it connects to the respective master and gateway nodes is transparent to the user.
2. This information is provided back to the Spark master node. At this point, we should be able to parse the query to determine which partitions (and their locations) within Cosmos DB we need to access.
3. This information is transmitted to the Spark worker nodes ...
4. Thus allowing the Spark worker nodes to connect directly to the Cosmos DB partitions directly to extract the data that is needed and bring the data back to the Spark partitions within the Spark worker nodes.

In [2]:
# Connection
flightsConfig = {
"Endpoint" : "https://pass-cosmosdb.documents.azure.com:443/",
"Masterkey" : "RMZXBrmg60L6tlwx52stSzt4r97WkZ2BXrI7rjuhRvZ5hiUcjiH6zmUoWeHLEqPhtv6mQE3gi8tqFOxfn97kMQ==",
"Database" : "flights",
"preferredRegions" : "West US; Central US",
"Collection" : "departuredelays", 
"SamplingRatio" : "1.0",
"schema_samplesize" : "1000",
"query_pagesize" : "200000",
"query_custom" : "SELECT c.date, c.delay, c.distance, c.origin, c.destination FROM c"
}

In [3]:
flights = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**flightsConfig).load()
flights.cache()
flights.createOrReplaceTempView("flights")
flights.count()

### Obtain Airport Information

In [5]:
# Set File Paths
airportsnaFilePath = "wasb://data@doctorwhostore.blob.core.windows.net/airport-codes-na.txt"

# Obtain airports dataset
airportsna = spark.read.csv(airportsnaFilePath, header='true', inferSchema='true', sep='\t')
airportsna.createOrReplaceTempView("airports")

### Flights departing from Seattle

In [7]:
%sql
select count(1) from flights where origin = 'SEA'

### Top 10 Delayed Destinations originating from Seattle

In [9]:
%sql
select a.city as destination, sum(f.delay) as TotalDelays, count(1) as Trips
from flights f
join airports a
  on a.IATA = f.destination
where f.origin = 'SEA'
and f.delay > 0
group by a.city 
order by sum(delay) desc limit 10

### Calculate median delays by destination cities departing from Seattle

In [11]:
%sql
select a.city as destination, percentile_approx(f.delay, 0.5) as median_delay
from flights f
join airports a
  on a.IATA = f.destination
where f.origin = 'SEA'
group by a.city 
order by percentile_approx(f.delay, 0.5)

## Building up a GraphFrames
Using GraphFrames for Apache Spark to run degree and motif queries against Cosmos DB

In [13]:
# Build `departureDelays` DataFrame
departureDelays = spark.sql("select cast(f.date as int) as tripid, cast(concat(concat(concat(concat(concat(concat('2014-', concat(concat(substr(cast(f.date as string), 1, 2), '-')), substr(cast(f.date as string), 3, 2)), ' '), substr(cast(f.date as string), 5, 2)), ':'), substr(cast(f.date as string), 7, 2)), ':00') as timestamp) as `localdate`, cast(f.delay as int), cast(f.distance as int), f.origin as src, f.destination as dst, o.city as city_src, d.city as city_dst, o.state as state_src, d.state as state_dst from flights f join airports o on o.iata = f.origin join airports d on d.iata = f.destination") 

# Create Temporary View and cache
departureDelays.createOrReplaceTempView("departureDelays")
departureDelays.cache()

In [14]:
# Note, ensure you have already installed the GraphFrames spack-package
from pyspark.sql.functions import *
from graphframes import *

# Create Vertices (airports) and Edges (flights)
tripVertices = airportsna.withColumnRenamed("IATA", "id").distinct()
tripEdges = departureDelays.select("tripid", "delay", "src", "dst", "city_dst", "state_dst")

# Cache Vertices and Edges
tripEdges.cache()
tripVertices.cache()

# Create TripGraph
tripGraph = GraphFrame(tripVertices, tripEdges)

### What flights departing SEA with the most significant average delays
Note, the joins are there to see the city name instead of the IATA codes.  The `rank()` code is there to help order the data correctly when viewed in Jupyter notebooks.

In [16]:
flightDelays = tripGraph.edges.filter("src = 'SEA' and delay > 0").groupBy("src", "dst").avg("delay").sort(desc("avg(delay)"))
flightDelays.createOrReplaceTempView("flightDelays")

In [17]:
%sql
select a.city, `avg(delay)` as avg_delay 
from flightDelays f
join airports a
on f.dst = a.iata
order by `avg(delay)` 
desc limit 10

### Which is the most important airport (in terms of connections)
It would take a relatively complicated SQL statement to calculate all of the edges to a single vertex, grouped by the vertices.  Instead, we can use the graph `degree` method.

In [19]:
airportConnections = tripGraph.degrees.sort(desc("degree"))
airportConnections.createOrReplaceTempView("airportConnections")

In [20]:
%sql
select a.city, f.degree 
from airportConnections f 
join airports a
  on a.iata = f.id
order by f.degree desc 
limit 10


### Are there direct flights between Seattle and San Jose?

In [22]:
filteredPaths = tripGraph.bfs(
    fromExpr = "id = 'SEA'",
    toExpr = "id = 'SJC'",
    maxPathLength = 1)
display(filteredPaths)

### But are there any direct flights between San Jose and Buffalo?
* Try maxPathLength = 1 which means one edge (i.e. one flight) between `SJC` and `BUF`, i.e. direct flight
* Try maxPathLength = 2 which means two edges between `SJC` and `BUF`, i.e. all the different variations of flights between San Jose and Buffalo with only one stop oever in between?

In [24]:
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SJC'",
  toExpr = "id = 'BUF'",
  maxPathLength = 1)
display(filteredPaths)

In [25]:
filteredPaths = tripGraph.bfs(
  fromExpr = "id = 'SJC'",
  toExpr = "id = 'BUF'",
  maxPathLength = 2)
display(filteredPaths)

### In that case, what is the most common transfer point between San Jose and Buffalo?

In [27]:
display(filteredPaths.groupBy("v1.id", "v1.City").count().orderBy(desc("count")).limit(10))

## Predicting Flight Delays
Extending upon analysis we have done up to this point, can we also predict if a flight will be delayed, on-time, or early based on the available data.

### Prepare the Dataset
The first thing we will do is to cleanse the data and apply some labels to our information (e.g. early, on-time, delayed).  As well, we will want to remove any rows with NULL values.

In [29]:
# This contains a generated mapping between tripid and airline
#   You can get the file at https://github.com/dennyglee/databricks/blob/master/misc/trip_airline_map.csv
#   For this example, the trip_airline_map.csv file has been pushed to in my mounted bucket.
tripAirlineMapFilePath = "wasb://data@doctorwhostore.blob.core.windows.net/trip_airline_map.csv"
tripAirlineMap = spark.read.csv(tripAirlineMapFilePath, sep=",", header=True)
tripAirlineMap.createOrReplaceTempView("tripAirlineMap")

In [30]:
# Prep dataset
# Including only Seattle and Las Vegas for this demo
flightML = spark.sql("select cast(distance as double) as distance, src as origin, state_src as origin_state, dst as destination, state_dst as destination_state, concat(concat(concat(cast(tripid as string), src), dst), cast((delay + 2000) as string)) as trip_identifier, case when delay <= 0 then 'on-time' else 'delayed' end as flight_status from departureDelays where src IN ('LAS', 'SEA')")
flightML = flightML.dropna().dropDuplicates()
flightML.createOrReplaceTempView("flightML")


In [31]:
# Join flights and airline information
dataset = spark.sql("select f.distance, f.origin, f.origin_state, f.destination, f.destination_state, f.trip_identifier, f.flight_status, m.airline from flightML f join tripAirlineMap m on m.trip_identifier = f.trip_identifier")
dataset = dataset.dropDuplicates()
#dataset = flightML
cols = dataset.columns

In [32]:
dataset.printSchema()

### Building ML Pipeline
Before we can run our various models against this data, we will first need to vectorize our data via One-Hot Encorder (for category data), String Indexer (create an index based on our labelled values), and Vector Assembler.

In [34]:
# One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["origin", "origin_state", "destination", "destination_state", "trip_identifier", "airline"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [stringIndexer, encoder]

# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "flight_status", outputCol = "label")
stages += [label_stringIdx]

# Transform all features into a vector using VectorAssembler
numericCols = ["distance"]
assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

In [35]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(dataset)
dataset = pipelineModel.transform(dataset)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
dataset = dataset.select(selectedcols)
display(dataset)

### Randomly split data into training and test datasets
* Set the seed for reproducibility

In [37]:
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed = 100)

## Logistic Regression
Let's try using logistic regression to see if we can accurately predict if a flight will be delayed.
* First, we will train the data using Logistic Regression
* Next we will run that model against the testData

In [39]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [40]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

### View LR Model's predictions
* Recall, label is the actual test value, prediction is the predicted value
 * where 0 - on-time, 1 - delayed

In [42]:
selected = predictions.select("label", "prediction", "probability", "flight_status", "destination", "destination_state").where("destination = 'SEA'")
display(selected)

#### Evaluate our model
Let's use the `BinaryClassificationEvaluator` to determine the precision of our model.

In [44]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

In [45]:
predictions.createOrReplaceTempView("predictions")

In [46]:
%sql
select * from predictions limit 10

In [47]:
%sql
select confusion, count(1)
from (
select 
case
  when label = 0 and prediction = 0 then 'True Positives (On-Time, Predicted: On-Time)'
  when label = 0 and prediction = 1 then 'False Negatives (On-Time, Predicted: Delayed)'
  when label = 1 and prediction = 0 then 'False Postitives (Delayed, Preidcted: On-Time)'
  when label = 1 and prediction = 1 then 'True Negatives (Delayed, Predicted: Delayed)'
end as confusion
from predictions
) a
group by confusion


### Plot decision boundary of the logistic regression model

In [49]:
# Take a sample of 200 to plot decision boundary of logistic regression
predictions_sample = predictions.sample(False, 0.1, seed=100).limit(200)
predictions_sample.createOrReplaceTempView("predictions_sample")

# Build schema for data
from pyspark.sql.types import *
schema = StructType([
        StructField('X1', FloatType(), True), 
        StructField('X2', StringType(), True),
        StructField('y', IntegerType(), True)])

# Extract data
ps = predictions_sample.rdd.map(lambda p: (float(p.rawPrediction[0]), p.airline, p.label) )

# Convert to DataFrame
psDF = ps.toDF(["X1", "X2", "y"], schema)
psDF.createOrReplaceTempView("psDF")

In [50]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt

#X, y = make_classification(200, 2, 2, 0, weights=[.5, .5], random_state=15)
X = np.asarray(pyX)
z = np.asarray(pyY, dtype=np.int)
y = z.ravel() 

clf = LogisticRegression().fit(X[:100], y[:100])

xx, yy = np.mgrid[-5:5:.01, -5:5:.01]
grid = np.c_[xx.ravel(), yy.ravel()]
probs = clf.predict_proba(grid)[:, 1].reshape(xx.shape)

f, ax = plt.subplots(figsize=(8, 6))
contour = ax.contourf(xx, yy, probs, 25, cmap="RdBu",
                      vmin=0, vmax=1)
ax_c = f.colorbar(contour)
ax_c.set_label("$P(y = 1)$")
ax_c.set_ticks([0, .25, .5, .75, 1])

ax.scatter(X[100:,0], X[100:, 1], c=y[100:], s=50,
           cmap="RdBu", vmin=-.2, vmax=1.2,
           edgecolor="white", linewidth=1)

ax.set(aspect="equal",
       xlim=(-5, 5), ylim=(-5, 5),
       xlabel="$X_1$", ylabel="$X_2$")